In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 38.3 MB/s 
     |████████████████████████████████| 182 kB 70.9 MB/s 
     |████████████████████████████████| 168 kB 68.4 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 168 kB 68.0 MB/s 
     |████████████████████████████████| 166 kB 69.4 MB/s 
     |████████████████████████████████| 166 kB 88.9 MB/s 
     |████████████████████████████████| 162 kB 84.1 MB/s 
     |████████████████████████████████| 162 kB 80.4 MB/s 
     |████████████████████████████████| 158 kB 57.8 MB/s 
     |████████████████████████████████| 157 kB 69.5 MB/s 
     |████████████████████████████████| 157 kB 47.1 MB/s 
     |████████████████████████████████| 157 kB 64.4 MB/s 
     |████████████████████████████████| 157 kB 13.8 MB/s 
     |████████████████████████████████| 157 kB 60.3 MB/s 
     |████████████████████████████████| 157 kB 62.5 MB/s 
     |████████████████████████████████| 157 kB 81.9 MB/s 
     |██████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.


In [6]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [7]:
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling='avg')
model_resnet50=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet50,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet50.trainable=False
model_resnet50.summary()

94765736/94765736 [==============================] - 5s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


In [8]:
model_resnet50.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [9]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet50",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [10]:
%%time
epochs=40
history = model_resnet50.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback(
    compute_flops = True,
    save_model = True
)]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.4075 - accuracy: 0.4276

wandb: Adding directory to artifact (/content/wandb/run-20221208_070219-22wvhn4o/files/model-best)... Done. 0.8s


22/22 [==============================] - 299s 9s/step - loss: 1.4075 - accuracy: 0.4276 - val_loss: 1.3170 - val_accuracy: 0.5429
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.2703 - accuracy: 0.5298

wandb: Adding directory to artifact (/content/wandb/run-20221208_070219-22wvhn4o/files/model-best)... Done. 0.8s


...

22/22 [==============================] - 26s 1s/step - loss: 0.9098 - accuracy: 0.6278 - val_loss: 0.9558 - val_accuracy: 0.6286
Epoch 36/40
22/22 [==============================] - 4s 159ms/step - loss: 0.9259 - accuracy: 0.6321 - val_loss: 0.9569 - val_accuracy: 0.5771
Epoch 37/40
22/22 [==============================] - ETA: 0s - loss: 0.9162 - accuracy: 0.6349

wandb: Adding directory to artifact (/content/wandb/run-20221208_070219-22wvhn4o/files/model-best)... Done. 0.9s


22/22 [==============================] - 28s 1s/step - loss: 0.9162 - accuracy: 0.6349 - val_loss: 0.9442 - val_accuracy: 0.5771
Epoch 38/40
22/22 [==============================] - 4s 158ms/step - loss: 0.9104 - accuracy: 0.6506 - val_loss: 0.9717 - val_accuracy: 0.5771
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.9145 - accuracy: 0.6562

wandb: Adding directory to artifact (/content/wandb/run-20221208_070219-22wvhn4o/files/model-best)... Done. 0.9s


22/22 [==============================] - 27s 1s/step - loss: 0.9145 - accuracy: 0.6562 - val_loss: 0.9403 - val_accuracy: 0.6114
Epoch 40/40
22/22 [==============================] - 4s 159ms/step - loss: 0.9076 - accuracy: 0.6435 - val_loss: 0.9527 - val_accuracy: 0.5771
CPU times: user 9min 3s, sys: 44.6 s, total: 9min 48s
Wall time: 17min 19s


In [11]:
wandb.alert(title="Run finished",
            text='ResNet50')

In [12]:
wandb.finish()

accuracy,▁▄▅▅▅▆▆▆▅▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇███
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▁▄▃▁▃▃▁▁▁▁▁▂▁▁▁▁▁▃▂▁▁▂▄▅▃▄▄▄▃▃▃▂█▄▄▄▇▄
val_loss,█▇▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁
GFLOPs,3.86226
accuracy,0.64347
best_epoch,38
best_val_loss,0.9403
epoch,39
loss,0.90758


In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv


name
Tesla T4
